In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from umap import UMAP
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt

from scipy.stats import zscore
from scipy.io import loadmat, savemat
import mat73
import os
import time

# conda activate shinlab # installed umap here
# DON'T FORGET: to use synology, need to run the following line in terminal
# net use S: \\shinlab\ShinLab
print('DON''T FORGET: to use synology, need to run in terminal: \"net use S: \\shinlab\ShinLab\" ')


C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DONT FORGET: to use synology, need to run in terminal: "net use S: \shinlab\ShinLab" 


In [2]:
"""
UMAP hyperparameters used were: 
    n_neighbors = 20, metric = 'cosine', output_metric ='euclidean', learning_rate = 1.0, init = 'spectral', 
    min_dist = 0.1, spread = 1.0, repulsion_strength = 1.0, negative_sample_rate = 5, target_metric = 'categorical', 
    dens_lambda = 2.0, dens_frac = 0.3, dens_var_shift=0.1.

Low-dimensional manifold visualization with UMAP (supervised)
With supervised methods, data from the same trial were given the same label. Data points
that share the same labels were leveraged so that similar points were embedded closer together.
For example, visualizations in Fig. 1G, H were produced by supervised UAMP dimensionality
reduction, where we exploited this option by using trial block labels in the behavioral data as
supervised information. This was implemented by passing the trial block number as target when
calling the fit_transform function from the UMAP python package
https://github.com/lmcinnes/umap/blob/master/doc/how_umap_works.rst.
"""    

"\nUMAP hyperparameters used were: \n    n_neighbors = 20, metric = 'cosine', output_metric ='euclidean', learning_rate = 1.0, init = 'spectral', \n    min_dist = 0.1, spread = 1.0, repulsion_strength = 1.0, negative_sample_rate = 5, target_metric = 'categorical', \n    dens_lambda = 2.0, dens_frac = 0.3, dens_var_shift=0.1.\n\nLow-dimensional manifold visualization with UMAP (supervised)\nWith supervised methods, data from the same trial were given the same label. Data points\nthat share the same labels were leveraged so that similar points were embedded closer together.\nFor example, visualizations in Fig. 1G, H were produced by supervised UAMP dimensionality\nreduction, where we exploited this option by using trial block labels in the behavioral data as\nsupervised information. This was implemented by passing the trial block number as target when\ncalling the fit_transform function from the UMAP python package\nhttps://github.com/lmcinnes/umap/blob/master/doc/how_umap_works.rst.\n"

In [3]:
# todo: add 10-fold cross validation and preprocessing (mean-centering/z-scoring) based on train set

def semi_supervised_umap_knn(data, labels, probedata, n_neighbors=20, n_components=2, n_splits=10, opt_supervise_umap=True, random_state=42):
    """
    Perform semi-supervised UMAP with kNN decoding, incorporating negative sampling and advanced validation.

    Parameters:
        data (numpy.ndarray): High-dimensional input data. NtrialsXNneurons
        labels (numpy.ndarray): Labels for supervised UMAP training. NtrialsX1
        n_neighbors (int): Number of neighbors for UMAP and kNN.
        n_components (int): Number of UMAP dimensions.
        test_size (float): Fraction of data for testing.
        random_state (int): Random state for reproducibility.

    Returns:
        dict: Results including accuracy, confusion matrix, and embeddings.
    """

    opt_meancenter_embeddings = False    
    
    # todo: preprocess (mean-centering/z-scoring) based on train set
    X_probe = probedata
    
    # # Split data into training and test sets
    # X_train, X_test, y_train, y_test = train_test_split(
    #     data, labels, test_size=1.0/n_splits, random_state=random_state)

    # # Cross-validation setup
    # kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    # Create StratifiedKFold object
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    train_index_list = []
    test_index_list = []
    y_test_list = []    
    accuracies = []
    confusion_matrices = []
    
    train_embeddings_list = []
    test_embeddings_list = []
    probe_embeddings_list = []
    
    y_train_pred_list = []
    y_test_pred_list = []
    y_probe_pred_list = []
    
    for train_index, test_index in skf.split(data, labels):
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
            
        # Apply supervised UMAP with adjusted hyperparameters
        umap_model = UMAP(
            n_neighbors=n_neighbors,
            n_components=n_components,
            metric="cosine", # note, this makes mean-centering unnecessary
            output_metric="euclidean",
            learning_rate=1.0,
            init="spectral",
            min_dist=0.1,
            spread=1.0,
            repulsion_strength=1.0,
            negative_sample_rate=5,
            target_metric="categorical",
            dens_lambda=2.0,
            dens_frac=0.3,
            dens_var_shift=0.1,
            random_state=random_state
        )
        
        if opt_supervise_umap:
            # make umap supervised
            embeddings = umap_model.fit_transform(X_train, y=y_train)
        else:
            embeddings = umap_model.fit_transform(X_train)
    
        # to make umap unsupervised, use code below instead
        # embeddings = umap_model.fit_transform(X_train)
        
        if opt_meancenter_embeddings:
            train_embeddings = embeddings.copy()
            train_embeddings = train_embeddings - np.mean(embeddings, axis=0)
        else:
            train_embeddings = embeddings.copy()
            
        # Train kNN classifier on UMAP embeddings
        knn = KNeighborsClassifier(n_neighbors=n_neighbors)
        knn.fit(train_embeddings, y_train)
    
        # Transform test data using the same UMAP model
        test_embeddings = umap_model.transform(X_test)
    
        if opt_meancenter_embeddings:
            test_embeddings = test_embeddings - np.mean(embeddings, axis=0)
        
        # Predict train/test labels
        y_train_pred = knn.predict(train_embeddings)
        y_test_pred = knn.predict(test_embeddings)
    
            
        # probe data
        probe_embeddings = umap_model.transform(X_probe)
        if opt_meancenter_embeddings:
            probe_embeddings = probe_embeddings - np.mean(embeddings, axis=0)
        y_probe_pred = knn.predict(probe_embeddings)
        
        # Compute accuracy and confusion matrix
        accuracy = accuracy_score(y_test, y_test_pred)
        conf_matrix = confusion_matrix(y_test, y_test_pred)

        
        train_index_list.append(train_index)
        test_index_list.append(test_index)    
        y_test_list.append(y_test)    
        accuracies.append(accuracy)
        confusion_matrices.append(conf_matrix)
        
        train_embeddings_list.append(train_embeddings)
        test_embeddings_list.append(test_embeddings)
        probe_embeddings_list.append(probe_embeddings)
                
        y_train_pred_list.append(y_train_pred)
        y_test_pred_list.append(y_test_pred)
        y_probe_pred_list.append(y_probe_pred)
        
    return {
        "train_index": train_index_list,
        "test_index": test_index_list,
        "test_truelabels": y_test_list,
        "accuracy": accuracies,
        "confusion_matrix": confusion_matrices,
        "train_embeddings": train_embeddings_list,
        "test_embeddings": test_embeddings_list,
        "probe_embeddings": probe_embeddings_list,
        "train_predlabels": y_train_pred_list,
        "test_predlabels": y_test_pred_list,
        "probe_predlabels": y_probe_pred_list,
    }


In [4]:
# use this code to decode individual sessions
'''
# whether to z-score all R data
optz = False

# Define the data directory
datadir = r'\\shinlab\ShinLab\OpenScopeData\00248_v240130\\'

# List all files and directories in the data directory
nwbdir = os.listdir(datadir)

# # Filter for sessions that contain 'sub-' or 'sub_'
# nwbsessions = [name for name in nwbdir if 'sub-' in name or 'sub_' in name]
# # Print the filtered sessions (optional)
# print(nwbsessions)

nwbsessions = ['sub-619293', 'sub-619296', 'sub-620333', 'sub-620334', 
    'sub-625545', 'sub-625554', 'sub-625555', 'sub-630506', 
    'sub-631510', 'sub-631570', 'sub-633229', 'sub-637484'];

ises = 2
pathpp = os.path.join(datadir, 'postprocessed', nwbsessions[ises], '')
print(pathpp)

fn2load = pathpp + 'postprocessed.mat'
try:
    postprocessed = loadmat(fn2load)
except:
    postprocessed = mat73.loadmat(fn2load)

# print("Keys in the file:", list(postprocessed.keys()))
Rall = postprocessed['Rall']
vis = postprocessed['vis']
neuallloc = postprocessed['neuallloc']

neuallloc = postprocessed['neuallloc']
neuV1 = np.array(['VISp' in loc[0] and 'VISpm' not in loc[0] for loc in neuallloc])

whichblock = 'ICwcfg1_presentations'

blocktrialorder = vis[whichblock]['ICtrialtypes'][vis[whichblock]['trialorder'].astype(int)]
trialsoi = np.isin(blocktrialorder, [106, 107, 110, 111])

Rblock = Rall[whichblock][:,neuV1].copy()
if optz:
    Rblock = (Rblock - np.mean(Rblock, axis=0))/np.std(Rblock, axis=0)
    Rblock[np.isnan(Rblock)] = 0

Rdata = Rblock[trialsoi,:].copy()
vislabel = blocktrialorder[trialsoi]

# print(Rblock.shape, Rdata.shape)

tic = time.time()
semiUMAPout = semi_supervised_umap_knn(Rdata, vislabel, Rblock)
toc = time.time - tic
print('Elapsed time: ', toc)

# unUMAPout = semi_supervised_umap_knn(Rdata, vislabel, Rblock, opt_supervise_umap=False)

fn2save = pathpp + 'UMAP_KNN_decoding_V1.mat'
savemat(fn2save, semiUMAPout)
'''

'\n# whether to z-score all R data\noptz = False\n\n# Define the data directory\ndatadir = r\'\\shinlab\\ShinLab\\OpenScopeData\x0248_v240130\\\'\n\n# List all files and directories in the data directory\nnwbdir = os.listdir(datadir)\n\n# # Filter for sessions that contain \'sub-\' or \'sub_\'\n# nwbsessions = [name for name in nwbdir if \'sub-\' in name or \'sub_\' in name]\n# # Print the filtered sessions (optional)\n# print(nwbsessions)\n\nnwbsessions = [\'sub-619293\', \'sub-619296\', \'sub-620333\', \'sub-620334\', \n    \'sub-625545\', \'sub-625554\', \'sub-625555\', \'sub-630506\', \n    \'sub-631510\', \'sub-631570\', \'sub-633229\', \'sub-637484\'];\n\nises = 2\npathpp = os.path.join(datadir, \'postprocessed\', nwbsessions[ises], \'\')\nprint(pathpp)\n\nfn2load = pathpp + \'postprocessed.mat\'\ntry:\n    postprocessed = loadmat(fn2load)\nexcept:\n    postprocessed = mat73.loadmat(fn2load)\n\n# print("Keys in the file:", list(postprocessed.keys()))\nRall = postprocessed[\'Rall\

In [ ]:
# Define the data directory
datadir = r'\\shinlab\ShinLab\OpenScopeData\00248_v240130\\'

# List all files and directories in the data directory
nwbdir = os.listdir(datadir)

nwbsessions = ['sub-619293', 'sub-619296', 'sub-620333', 'sub-620334', 
    'sub-625545', 'sub-625554', 'sub-625555', 'sub-630506', 
    'sub-631510', 'sub-631570', 'sub-633229', 'sub-637484'];

for ises in np.arange(10,Nslopes,2):
    pathpp = os.path.join(datadir, 'postprocessed', nwbsessions[ises], '')
    print(ises, pathpp)
    
    fn2load = pathpp + 'spkcnt_ICwcfg1_hireptt_V1RS_lmlv.mat'
    try:
        spkcntlmlv = loadmat(fn2load)
    except:
        spkcntlmlv = mat73.loadmat(fn2load)
    
    
    blocktrialorder = spkcntlmlv['trialorder'].flatten()
    trialsoi = np.isin(blocktrialorder, [106, 107, 110, 111])
    vislabel = blocktrialorder[trialsoi]
    
    Rblock = spkcntlmlv['spkcntorig']
    Rdata = Rblock[trialsoi,:].copy()
    
    tic = time.time()
    UMAPorig = semi_supervised_umap_knn(Rdata, vislabel, Rblock)
    toc = time.time() - tic
    print('Elapsed time: ', toc)
    
    UMAPlmlv_list = []
    Nslopes = len(spkcntlmlv['disperses'][0])
    for islope in np.arange(Nslopes):
        Rblock = spkcntlmlv['spkcntlmlv'][0][islope]
        Rblock[np.isnan(Rblock)] = 0
        Rdata = Rblock[trialsoi,:].copy()
    
        tic = time.time()
        UMAPslope = semi_supervised_umap_knn(Rdata, vislabel, Rblock)
        toc = time.time() - tic
        print('Elapsed time: ', toc)
    
        UMAPlmlv_list.append(UMAPslope)
    
    UMAP_dict = {
        'UMAPorig': UMAPorig,
        'UMAPlmlv': UMAPlmlv_list
    }
    
    fn2save = pathpp + 'UMAP_KNN_decoding_V1RS_lmlvslopes.mat'
    savemat(fn2save, UMAP_dict)


In [ ]:
# visualize UMAP embeddings and KNN decoding results

UMAPout = semiUMAPout
# Output results
print("Accuracy:", np.mean(UMAPout["accuracy"]))
print("Confusion Matrix:\n", sum(UMAPout["confusion_matrix"])/len(UMAPout["confusion_matrix"]) )

# Visualize embeddings
plt.scatter(
    UMAPout["train_embeddings"][0][:, 0],
    UMAPout["train_embeddings"][0][:, 1],
    c=vislabel[: len(UMAPout["train_embeddings"][0])],
    cmap="viridis",
    s=10
)
plt.colorbar()
plt.title("UMAP Embeddings (Train)")
plt.show()

plt.scatter(
    UMAPout["test_embeddings"][0][:, 0],
    UMAPout["test_embeddings"][0][:, 1],
    c=UMAPout["test_truelabels"][0],
    cmap="viridis",
    s=10
)
plt.colorbar()
plt.title("UMAP Embeddings (Test)")
plt.show()


In [8]:
print( np.arange(10,Nslopes,2))
for islope in np.arange(10,Nslopes,2):
    print(spkcntlmlv['disperses'][0][islope])

[10 12 14 16 18 20 22 24 26 28 30]
0.0
0.20000000000000018
0.40000000000000013
0.5999999999999999
0.7999999999999998
1.0
1.2
1.4
1.6
1.8
2.0


In [9]:
datadir = r'\\shinlab\ShinLab\OpenScopeData\00248_v240130\\'

# List all files and directories in the data directory
nwbdir = os.listdir(datadir)

nwbsessions = ['sub-619293', 'sub-619296', 'sub-620333', 'sub-620334', 
    'sub-625545', 'sub-625554', 'sub-625555', 'sub-630506', 
    'sub-631510', 'sub-631570', 'sub-633229', 'sub-637484'];

for ises in np.arange(len(nwbsessions)):
    pathpp = os.path.join(datadir, 'postprocessed', nwbsessions[ises], '')
    print(ises, pathpp)
    
    fn2load = pathpp + 'spkcnt_ICwcfg1_hireptt_V1RS_lmlv.mat'
    try:
        spkcntlmlv = loadmat(fn2load)
    except:
        spkcntlmlv = mat73.loadmat(fn2load)
    
    blocktrialorder = spkcntlmlv['trialorder'].flatten()
    trialsoi = np.isin(blocktrialorder, [106, 107, 110, 111])
    vislabel = blocktrialorder[trialsoi]
    
    Rblock = spkcntlmlv['spkcntorig']
    Rdata = Rblock[trialsoi,:].copy()
    
    Nsplits = 10
    Ndim = 5
    tic = time.time()
    UMAPorig = semi_supervised_umap_knn(Rdata, vislabel, Rblock, n_components=Ndim, n_splits=Nsplits, opt_supervise_umap=False)
    toc = time.time() - tic
    print('Elapsed time: ', toc)
    
    UMAPlmlv_list = []
    Nslopes = len(spkcntlmlv['disperses'][0])
    for islope in np.arange(10,Nslopes,2):
        Rblock = spkcntlmlv['spkcntlmlv'][0][islope]
        Rblock[np.isnan(Rblock)] = 0
        Rdata = Rblock[trialsoi,:].copy()
    
        tic = time.time()
        UMAPslope = semi_supervised_umap_knn(Rdata, vislabel, Rblock, n_components=Ndim, n_splits=Nsplits, opt_supervise_umap=False)
        toc = time.time() - tic
        print('Elapsed time: ', toc)
    
        UMAPlmlv_list.append(UMAPslope)
    
    UMAP_dict = {
        'UMAPorig_unsup5dim': UMAPorig,
        'UMAPlmlv_unsup5dim': UMAPlmlv_list
    }
    
    fn2save = pathpp + 'UMAP_KNN_decoding_V1RS_lmlvslopes_unsupervised5dim.mat'
    savemat(fn2save, UMAP_dict)    


0 \\shinlab\ShinLab\OpenScopeData\00248_v240130\\postprocessed\sub-619293\


C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWar

Elapsed time:  175.87420678138733


C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWar

Elapsed time:  177.87224125862122


...

Elapsed time:  175.21188974380493


C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWar

Elapsed time:  175.68071866035461


C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\ProgramData\anaconda3\envs\hs13\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWar

Elapsed time:  174.1750156879425
